In [28]:
import psycopg2
from pathlib import Path
import pandas as pd
import environ
import os

In [29]:
# Enviroment variables
BASE_DIR = Path('.env').resolve()

env = environ.Env()
environ.Env.read_env(BASE_DIR)

# Database connection
conn = psycopg2.connect(
    host = env('DB_HOST'),
    database = env('DB_NAME'),
    user = env('DB_USER'),
    password = env('DB_PASSWORD'),
    port = env('DB_PORT')
)

cursor = conn.cursor()
print(cursor)

<cursor object at 0x74874ebc3100; closed: 0>


In [30]:
# Datos para subir
datos_dir = os.path.expanduser("/home/edu/edu/datos")

file = 'datos.csv'

datos = pd.read_csv(os.path.join(datos_dir, file))
datos.head()

,fecha,monto,categoría,cuenta,presupuesto,descripción,moneda
0,05/12/2018,23.79,Inicio,Efectivo,NaN,NaN,Dólar estadounidense
1,05/12/2018,43.81,Inicio,Casa,NaN,NaN,Dólar estadounidense
2,05/12/2018,227.73,Inicio,Edu 6367170,NaN,NaN,Dólar estadounidense
3,05/12/2018,24.41,Inicio,Produbanco,NaN,NaN,Dólar estadounidense
4,05/12/2018,30.00,Inicio,Edu 75032955,NaN,NaN,Dólar estadounidense


Currencies

In [31]:
# Currencies
monedas = datos['moneda'].unique()

data_to_insert = [
    (monedas[0], 'USD', 1, True, True),
    (monedas[1], 'ARG', 983.73,  False, True),
    (monedas[2], 'COP', 4289.14, False, True)
]

insert_query = """
INSERT INTO "tracker_currencies" (name, code, exchange_rate, principal, is_active)
VALUES (%s, %s, %s, %s, %s)
"""

#cursor.executemany(insert_query, data_to_insert)
cursor.executemany(insert_query, data_to_insert)
conn.commit()

UniqueViolation: duplicate key value violates unique constraint "tracker_currencies_code_key"
DETAIL:  Key (code)=(USD) already exists.


Accounts

In [32]:
datos['fecha'] = pd.to_datetime(datos['fecha'], dayfirst=True)
datos['monto'] = datos['monto'].str.replace(',', '').astype(float).round(2)

In [33]:
cuentas = datos['cuenta'].unique()
cuentas

array(['Efectivo', 'Casa', 'Edu 6367170', 'Produbanco ', 'Edu 75032955',
       'Colección', 'Fondo', 'B&G Trading', 'Visa 538', 'Mercado pago',
       'Visa 0878', 'Santander', 'Diners 9521', 'Diners 4920',
       'Cuenta de mama', 'IDB Federal Bank', 'Banco del Pacífico'],
      dtype=object)

In [34]:
created_date = []
for x in range(len(cuentas)):
    created_date.append(datos[datos['cuenta'] == cuentas[x]]['fecha'].min())

last_updated = []
for x in range(len(cuentas)):
    last_updated.append(datos[datos['cuenta'] == cuentas[x]]['fecha'].max())

balance_cuenta = []
for x in range(len(cuentas)):
    balance_cuenta.append(datos[(datos['cuenta'] == cuentas[x]) & (datos['moneda'] == 'Dólar estadounidense')]['monto'].sum().round(2))

monedas = [1,1,1,1,1,1,1,1,1,2,1,2,1,1,1,1,1]

In [35]:
data_to_insert = []
for x in range(len(cuentas)):
    data_to_insert.append(
        (1, cuentas[x], 'checking', created_date[x], float(balance_cuenta[x]), monedas[x], last_updated[x])
    )

In [36]:
insert_query = """
INSERT INTO "tracker_accounts" (user_id, name, account_type, created_date, balance, currency_id, last_updated)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

#cursor.executemany(insert_query, data_to_insert)
cursor.executemany(insert_query, data_to_insert)
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


Categories

In [37]:
categorias = datos['categoría'].unique()

In [38]:
data_to_insert = []
for x in range(len(categorias)):
    data_to_insert.append(
        (1, categorias[x])
    )

In [39]:
conn.rollback()

In [40]:
insert_query = """
INSERT INTO "tracker_categories" (user_id, name)
VALUES (%s, %s)
"""

#cursor.executemany(insert_query, data_to_insert)
cursor.executemany(insert_query, data_to_insert)
conn.commit()

Transactions

In [41]:
cuentas_tr = {}
indices = range(3, 20)
for x in range(len(cuentas)):
    cuentas_tr[cuentas[x]] = indices[x]

In [42]:
datos['cuenta_tr'] = datos['cuenta'].apply(lambda x: cuentas_tr[x])

In [43]:
datos['tr_type'] = datos['monto'].apply(lambda x: 'income' if x > 0 else 'expense')

In [44]:
insert_query = """
SELECT * FROM "tracker_categories"
"""

#cursor.executemany(insert_query, data_to_insert)
cursor.execute(insert_query)
catg = cursor.fetchall()

In [45]:
catgs = {}
for x in range(len(catg)):
    catgs[catg[x][1]] = catg[x][0]

In [46]:
datos['catg_tr'] = datos['categoría'].apply(lambda x: catgs[x])

In [47]:
monedas_tr = {"Dólar estadounidense":1,
              "Pesos argentinos":2,
              "Pesos colombianos":3}

In [48]:
datos['moneda_tr'] = datos['moneda'].apply(lambda x: monedas_tr[x])

In [49]:
datos['descripción'] = datos['descripción'].astype(object)
datos['descripción'] = datos['descripción'].where(datos['descripción'].notna(), None)

In [50]:
data_to_insert = []
for x in range(len(datos)):
    data_to_insert.append(
        (int(datos['cuenta_tr'][x]), datos['tr_type'][x], datos['fecha'][x],
         datos['fecha'][x], float(datos['monto'][x]), datos['descripción'][x],
         int(datos['catg_tr'][x]), int(datos['moneda_tr'][x]), 1)
    )

In [ ]:
data_to_insert

[(3,
  'income',
  Timestamp('2018-12-05 00:00:00'),
  Timestamp('2018-12-05 00:00:00'),
  23.79,
  None,
  56,
  1,
  1),
 (4,
  'income',
  Timestamp('2018-12-05 00:00:00'),
  Timestamp('2018-12-05 00:00:00'),
  43.81,
  None,
  56,
  1,
  1),
 (5,
  'income',
  Timestamp('2018-12-05 00:00:00'),
  Timestamp('2018-12-05 00:00:00'),
  227.73,
  None,
  56,
  1,
  1),
 (6,
  'income',
  Timestamp('2018-12-05 00:00:00'),
  Timestamp('2018-12-05 00:00:00'),
  24.41,
  None,
  56,
  1,
  1),
 (7,
  'income',
  Timestamp('2018-12-05 00:00:00'),
  Timestamp('2018-12-05 00:00:00'),
  30.0,
  None,
  56,
  1,
  1),
 (3,
  'expense',
  Timestamp('2018-12-06 00:00:00'),
  Timestamp('2018-12-06 00:00:00'),
  -0.25,
  None,
  57,
  1,
  1),
 (3,
  'expense',
  Timestamp('2018-12-06 00:00:00'),
  Timestamp('2018-12-06 00:00:00'),
  -0.25,
  None,
  58,
  1,
  1),
 (3,
  'expense',
  Timestamp('2018-12-06 00:00:00'),
  Timestamp('2018-12-06 00:00:00'),
  -2.0,
  None,
  59,
  1,
  1),
 (3,
  'expens

In [53]:
data_to_insert[0]

(3,
 'income',
 Timestamp('2018-12-05 00:00:00'),
 Timestamp('2018-12-05 00:00:00'),
 23.79,
 None,
 56,
 1,
 1)

In [65]:
conn.rollback()

In [68]:
insert_query = """
INSERT INTO "tracker_transactions" (account_id, transaction_type, transaction_date,
created_date, amount, description, category_id, currency_id, user_id)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

#cursor.executemany(insert_query, data_to_insert)
try:
    cursor.executemany(insert_query, data_to_insert[1:])
    conn.commit()
except Exception as e:
    print("Error:", e)
    conn.rollback()
